### **Import necessary libraries**

In [ ]:
import numpy as np
import pandas as pd
import os
import re
from collections import OrderedDict
from math import factorial
from copy import deepcopy
import pickle
import random
import skimage.io as sk
from skimage import img_as_ubyte
from skimage.io import imread
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
import cv2
import numpy as np
from PIL import Image
from keras import backend as K
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten,Activation
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.models import Model,load_model, model_from_json
import tensorflow as tf
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization


### **Define the Forgeries Signature**

In [ ]:
image1 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/forgeries_1_1.png")
image2 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/forgeries_1_20.png")
image3 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/forgeries_1_21.png")
image4 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/forgeries_1_22.png")
image5 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/forgeries_1_23.png")
image6 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/forgeries_11_1.png")
image7 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/forgeries_11_20.png")
image8 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/forgeries_11_21.png")
image9 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/forgeries_11_22.png")
image10 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/forgeries_11_23.png")

fig, ax = plt.subplots(2,5, figsize = (15,10))
plt.subplots_adjust(hspace=0)

ax[0,0].imshow(image1)
ax[0,0].set_title("Forge_1")
ax[0,1].imshow(image2)
ax[0,1].set_title("Forge_20")
ax[0,2].imshow(image3)
ax[0,2].set_title("Forge_21")
ax[0,3].imshow(image4)
ax[0,3].set_title("Forge_22")
ax[0,4].imshow(image5)
ax[0,4].set_title("Forge_23")
ax[1,0].imshow(image6)
ax[1,0].set_title("Forge_1")
ax[1,1].imshow(image7)
ax[1,1].set_title("Forge_20")
ax[1,2].imshow(image8)
ax[1,2].set_title("Forge_21")
ax[1,3].imshow(image9)
ax[1,3].set_title("Forge_22")
ax[1,4].imshow(image10)
ax[1,4].set_title("Forge_23")

### **Preprocess the forgeries image**

In [ ]:
import cv2

def preprocess_image(image):
    # Convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the image
    img = cv2.resize(image, (256, 128))
    # Apply Gaussian Blur to remove noise
    img = cv2.GaussianBlur(img, (5, 5), 0)
    # Apply adaptive thresholding
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    return img

preprocessed_image1 = preprocess_image(image1)
preprocessed_image2 = preprocess_image(image2)
preprocessed_image3 = preprocess_image(image3)
preprocessed_image4 = preprocess_image(image4)
preprocessed_image5 = preprocess_image(image5)
preprocessed_image6 = preprocess_image(image6)
preprocessed_image7 = preprocess_image(image7)
preprocessed_image8 = preprocess_image(image8)
preprocessed_image9 = preprocess_image(image9)
preprocessed_image10 = preprocess_image(image10)

fig, ax = plt.subplots(2,5, figsize = (15,10))
plt.subplots_adjust(hspace=0)

ax[0,0].imshow(preprocessed_image1, cmap='gray')
ax[0,0].set_title("Forge_1")
ax[0,1].imshow(preprocessed_image2, cmap='gray')
ax[0,1].set_title("Forge_20")
ax[0,2].imshow(preprocessed_image3, cmap='gray')
ax[0,2].set_title("Forge_21")
ax[0,3].imshow(preprocessed_image4, cmap='gray')
ax[0,3].set_title("Forge_22")
ax[0,4].imshow(preprocessed_image5, cmap='gray')
ax[0,4].set_title("Forge_23")
ax[1,0].imshow(preprocessed_image6, cmap='gray')
ax[1,0].set_title("Forge_1")
ax[1,1].imshow(preprocessed_image7, cmap='gray')
ax[1,1].set_title("Forge_20")
ax[1,2].imshow(preprocessed_image8, cmap='gray')
ax[1,2].set_title("Forge_21")
ax[1,3].imshow(preprocessed_image9, cmap='gray')
ax[1,3].set_title("Forge_22")
ax[1,4].imshow(preprocessed_image10, cmap='gray')
ax[1,4].set_title("Forge_23")

### **Define the original images**

In [ ]:
image11 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/original_1_1.png")
image12 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/original_1_20.png")
image13 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/original_1_21.png")
image14 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/original_1_22.png")
image15 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_1/original_1_23.png")
image16 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/original_11_1.png")
image17 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/original_11_20.png")
image18 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/original_11_21.png")
image19 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/original_11_22.png")
image20 = sk.imread("/content/drive/MyDrive/Data/signatures/signatures_11/original_11_23.png")

fig, ax = plt.subplots(2,5, figsize = (15,10))
plt.subplots_adjust(hspace=0)

ax[0,0].imshow(image11)
ax[0,0].set_title("Real_1")
ax[0,1].imshow(image12)
ax[0,1].set_title("Real_20")
ax[0,2].imshow(image13)
ax[0,2].set_title("Real_21")
ax[0,3].imshow(image14)
ax[0,3].set_title("Real_22")
ax[0,4].imshow(image15)
ax[0,4].set_title("Real_23")
ax[1,0].imshow(image16)
ax[1,0].set_title("Real_1")
ax[1,1].imshow(image17)
ax[1,1].set_title("Real_20")
ax[1,2].imshow(image18)
ax[1,2].set_title("Real_21")
ax[1,3].imshow(image19)
ax[1,3].set_title("Real_22")
ax[1,4].imshow(image20)
ax[1,4].set_title("Real_23")

### **Preproecess the Original image**

In [ ]:
def preprocess_image(image):
    # Convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize the image
    img = cv2.resize(image, (256, 128))
    # Apply Gaussian Blur to remove noise
    img = cv2.GaussianBlur(img, (5, 5), 0)
    # Apply adaptive thresholding
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    return img

preprocessed_image11 = preprocess_image(image11)
preprocessed_image12 = preprocess_image(image12)
preprocessed_image13 = preprocess_image(image13)
preprocessed_image14 = preprocess_image(image14)
preprocessed_image15 = preprocess_image(image15)
preprocessed_image16 = preprocess_image(image16)
preprocessed_image17 = preprocess_image(image17)
preprocessed_image18 = preprocess_image(image18)
preprocessed_image19 = preprocess_image(image19)
preprocessed_image20 = preprocess_image(image20)

fig, ax = plt.subplots(2,5, figsize = (15,10))
plt.subplots_adjust(hspace=0)

ax[0,0].imshow(preprocessed_image11, cmap='gray')
ax[0,0].set_title("Real_1")
ax[0,1].imshow(preprocessed_image12, cmap='gray')
ax[0,1].set_title("Real_20")
ax[0,2].imshow(preprocessed_image13, cmap='gray')
ax[0,2].set_title("Real_21")
ax[0,3].imshow(preprocessed_image14, cmap='gray')
ax[0,3].set_title("Real_22")
ax[0,4].imshow(preprocessed_image15, cmap='gray')
ax[0,4].set_title("Real_23")
ax[1,0].imshow(preprocessed_image16, cmap='gray')
ax[1,0].set_title("Real_1")
ax[1,1].imshow(preprocessed_image17, cmap='gray')
ax[1,1].set_title("Real_20")
ax[1,2].imshow(preprocessed_image18, cmap='gray')
ax[1,2].set_title("Real_21")
ax[1,3].imshow(preprocessed_image19, cmap='gray')
ax[1,3].set_title("Real_22")
ax[1,4].imshow(preprocessed_image20, cmap='gray')
ax[1,4].set_title("Real_23")

### **Structure the images as geninue and forgery**

In [ ]:
def organize_preprocessed_data():
    """
    Organize your preprocessed images into proper data structure
    """
    genuine_signatures = [
        preprocessed_image11, preprocessed_image12, preprocessed_image13,
        preprocessed_image14, preprocessed_image15, preprocessed_image16,
        preprocessed_image17, preprocessed_image18, preprocessed_image19,
        preprocessed_image20
    ]

    forgery_signatures = [
        preprocessed_image1, preprocessed_image2, preprocessed_image3,
        preprocessed_image4, preprocessed_image5, preprocessed_image6,
        preprocessed_image7, preprocessed_image8, preprocessed_image9,
        preprocessed_image10
    ]

    return genuine_signatures, forgery_signatures

### **For SNN we need to feed two inputs we need to pair the images**

In [ ]:
import numpy as np

def create_signature_pairs(genuine_signatures, forgery_signatures):
    pairs = []
    labels = []

    # Positive pairs (genuine-genuine)
    for i in range(len(genuine_signatures)):
        for j in range(i+1, len(genuine_signatures)):
            pairs.append([genuine_signatures[i], genuine_signatures[j]])
            labels.append(1)  # same person

    # Negative pairs (genuine-forgery)
    for i in range(len(genuine_signatures)):
        pairs.append([genuine_signatures[i], forgery_signatures[i]])
        labels.append(0)  # different (forged)

    pairs = np.array(pairs)
    labels = np.array(labels)

    indices = np.arange(len(pairs))
    np.random.shuffle(indices)
    pairs = pairs[indices]
    labels = labels[indices]

    # Split into two arrays: X1 and X2 for Siamese input
    X1 = pairs[:, 0]
    X2 = pairs[:, 1]

    return X1, X2, labels

### **Spit the data and train**

In [ ]:
from sklearn.model_selection import train_test_split

# Load and pair your images
genuine, forgery = organize_preprocessed_data()
X1, X2, y = create_signature_pairs(genuine, forgery)

# Normalize
X1 = X1.astype('float32') / 255.0
X2 = X2.astype('float32') / 255.0

# Train-test split (with stratify for balanced classes)
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    X1, X2, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
print("Train:", np.unique(y_train, return_counts=True))
print("Test:", np.unique(y_test, return_counts=True))


### **Create a base CNN and define the similarty method**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
import tensorflow.keras.backend as K

def create_base_cnn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (10, 10), activation='relu')(inputs)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)
    return Model(inputs, x)

# Define L1 Distance function
def l1_distance(vectors):
    x, y = vectors
    return K.abs(x - y)

def build_siamese_network(input_shape):
    base_cnn = create_base_cnn(input_shape)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    encoded_a = base_cnn(input_a)
    encoded_b = base_cnn(input_b)

    # Calculate L1 distance between the encoded vectors
    l1_layer = Lambda(l1_distance)([encoded_a, encoded_b])

    # Add the output layer
    output = Dense(1, activation='sigmoid')(l1_layer)

    model = Model(inputs=[input_a, input_b], outputs=output)
    return model

### **Fit the model(Train)**

In [ ]:
model = build_siamese_network((128, 256, 1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit([X1_train, X2_train], y_train,
                    batch_size=8, epochs=10, validation_split=0.1)


In [ ]:
import numpy as np
print(np.unique(y_test, return_counts=True))

### **Checking the accuracy**

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

# Accuracy
loss, acc = model.evaluate([X1_test, X2_test], y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

# ROC AUC + F1
y_pred = model.predict([X1_test, X2_test])
print("ROC AUC:", roc_auc_score(y_test, y_pred))
print(classification_report(y_test, (y_pred > 0.5).astype(int)))

### **Validating the ouput**

In [ ]:
def verify_signature(preprocessed_image1, preprocessed_image2, model, threshold=0.6):
    sig1 = np.expand_dims(preprocessed_image1, axis=0)
    sig1 = np.expand_dims(sig1, axis=-1)
    sig2 = np.expand_dims(preprocessed_image13, axis=0)
    sig2 = np.expand_dims(sig2, axis=-1)

    score = model.predict([sig1, sig2])[0][0]
    print(f"Similarity Score: {score:.4f}")
    print("✅ Genuine" if score < threshold else "❌ Forged")
verify_signature(preprocessed_image1, preprocessed_image2, model)

### **Define the Custom objects**

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
import tensorflow as tf

# Euclidean Distance function used in Siamese Networks
def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

# Contrastive Loss Function
def contrastive_loss(y_true, y_pred):
    margin = 1.0
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


### **Save the model(Locally) for validation**

In [ ]:
model.save('siamese_model2.h5')


### **Checking the ouput**

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Define and register the l1_distance function
@tf.keras.utils.register_keras_serializable()
def l1_distance(vects):
    x, y = vects
    return K.abs(x - y)

# Define contrastive loss (if used in your model)
@tf.keras.utils.register_keras_serializable()
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Optional: Euclidean distance function (if used)
@tf.keras.utils.register_keras_serializable()
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def verify_signature_from_path_with_display(image_path1, image_path2, model_path, threshold=0.5):
    # Load the trained Siamese model with custom objects
    model = load_model(model_path, custom_objects={
        'contrastive_loss': contrastive_loss,
        'l1_distance': l1_distance,
        'euclidean_distance': euclidean_distance,
        'accuracy': 'accuracy'
    })

    # Display the original signature images
    img1_display = cv2.imread(image_path1)
    img2_display = cv2.imread(image_path2)

    if img1_display is None or img2_display is None:
        print("❌ Error: Could not load one or both images. Check the file paths.")
        return

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(cv2.cvtColor(img1_display, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Signature 1')
    ax[0].axis('off')
    ax[1].imshow(cv2.cvtColor(img2_display, cv2.COLOR_BGR2RGB))
    ax[1].set_title('Signature 2')
    ax[1].axis('off')
    plt.show()

    # Internal preprocessing function
    def preprocess_image(image_path):
        img = image.load_img(image_path, target_size=(128, 256), color_mode="grayscale")
        img_array = image.img_to_array(img)
        img_array = cv2.adaptiveThreshold(img_array.astype(np.uint8), 255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension
        img_array = img_array / 255.0  # Normalize
        return img_array

    # Preprocess both images
    sig1 = preprocess_image(image_path1)
    sig2 = preprocess_image(image_path2)

    sig1 = np.expand_dims(sig1, axis=0)  # Add batch dimension
    sig2 = np.expand_dims(sig2, axis=0)

    # Predict similarity
    score = model.predict([sig1, sig2])[0][0]
    print(f"\nSimilarity Score: {score:.4f}")
    print("✅ Genuine" if score <= threshold else "❌ Forged")

# Example usage
verify_signature_from_path_with_display(
    '/content/drive/MyDrive/Data/signatures/signatures_15/original_15_1.png',
    '/content/drive/MyDrive/Data/signatures/signatures_1/original_1_1.png',
    'siamese_model2.h5'
)


### **Saving the Final model(drive)**

In [ ]:
model.save('/content/drive/MyDrive/Data/saved_model/siamese_model2.h5')


### **Final prediction whether the two inputs are geninue or forged with reasons**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import cv2

def verify_signature(image_path1, image_path2, model, threshold=0.6):
    """
    Parameters:
        image_path1 (str): Path to first signature image
        image_path2 (str): Path to second signature image
        model (keras.Model): The trained Siamese model
        threshold (float): Decision threshold for similarity score
    """

    # 2. Load and preprocess the two images
    def preprocess_image(img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 128))
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=-1)
        img = np.expand_dims(img, axis=0)
        return img

    sig1 = preprocess_image(image_path1)
    sig2 = preprocess_image(image_path2)

    # 3. Display the images
    img1_show = cv2.imread(image_path1)
    img2_show = cv2.imread(image_path2)

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img1_show, cv2.COLOR_BGR2RGB))
    plt.title("Signature 1")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(img2_show, cv2.COLOR_BGR2RGB))
    plt.title("Signature 2")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

    # 4. Predict similarity score
    score = model.predict([sig1, sig2])[0][0]
    print(f"Similarity Score: {score:.4f}")
    if score < threshold:
        print("✅ Genuine Signature")
        print("Reason:\n"
      "1. Natural Flow – The pen pressure, stroke direction, and speed match the person’s usual hand movement.\n"
      "2. Consistent Style – Letter shapes, slant, and spacing are uniform with past verified samples.\n"
      "3. Muscle Memory – Genuine signatures come from repeated use; the motion is subconscious and hard to replicate.\n"
      "4. Stable Pressure – Original signatures show consistent ink pressure and pen lifts, unlike hesitations in fakes.\n"
      "5. Verified History – Matches with stored digital or physical records used in banking or legal contexts.")

    else:
        print("❌ Forged Signature")
        print("Reason:\n"
      "1. Shaky Strokes – Forgers often hesitate, causing unnatural or wobbly lines.\n"
      "2. Inconsistent Size – Letters or signature size doesn’t align with authentic records.\n"
      "3. Wrong Pen Pressure – Forgeries usually show uneven or forced pen pressure.\n"
      "4. Timing Mismatch – Genuine signatures take milliseconds; fakes take longer and lack fluid motion.\n"
      "5. Mismatch with Model – AI models trained on hundreds of real samples detect micro-pattern mismatches no human eye can spot.")


### **Final ouput**

In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from keras.saving import register_keras_serializable

@register_keras_serializable()
def euclidean_distance(vectors):
    x, y = vectors
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

@register_keras_serializable()
def l1_distance(vectors):
    x, y = vectors
    return K.abs(x - y)

# Load the model with the custom objects
model = load_model("/content/drive/MyDrive/Data/saved_model/siamese_model2.h5",
                   custom_objects={'euclidean_distance': euclidean_distance,
                                   'l1_distance': l1_distance})

# Now you can use the verify_signature function
verify_signature("/content/drive/MyDrive/Data/signatures/signatures_16/original_16_22.png",
                 "/content/drive/MyDrive/Data/signatures/signatures_16/original_16_22.png",
                 model)


In [ ]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from keras.saving import register_keras_serializable

@register_keras_serializable()
def euclidean_distance(vectors):
    x, y = vectors
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

@register_keras_serializable()
def l1_distance(vectors):
    x, y = vectors
    return K.abs(x - y)

# Load the model with the custom object
model = load_model("/content/drive/MyDrive/Data/saved_model/siamese_model2.h5", custom_objects={'euclidean_distance': euclidean_distance,'l1_distance': l1_distance})

# Now you can use the verify_signature function
verify_signature("/content/drive/MyDrive/Data/signatures/signatures_12/original_12_22.png", "/content/drive/MyDrive/Data/signatures/signatures_12/original_12_22.png", model)

### **Code for Extracting Images From document**

In [ ]:
import cv2
import numpy as np
import os

def pad_image(img, padding=30):
    return cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=[255, 255, 255])

def extract_signatures(image_path):
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Unable to load image: {image_path}")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height = img.shape[0]

    # Directories
    freehand_dir = "freehand_signatures"
    boxed_dir = "boxed_signatures"
    os.makedirs(freehand_dir, exist_ok=True)
    os.makedirs(boxed_dir, exist_ok=True)

    # Freehand Signature Detection
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    freehand_count = 0
    padding = 30

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)

        if 100 < w < 800 and 30 < h < 300 and 1.0 < aspect_ratio < 10.0:
            if y < height * 0.5:
                continue

            # Expand bounding box with padding
            x_exp = max(x - padding, 0)
            y_exp = max(y - padding, 0)
            w_exp = min(w + 2 * padding, img.shape[1] - x_exp)
            h_exp = min(h + 2 * padding, img.shape[0] - y_exp)

            # Draw expanded bounding box in blue
            cv2.rectangle(img, (x_exp, y_exp), (x_exp + w_exp, y_exp + h_exp), (255, 0, 0), 4)

            # Crop and save the padded region
            signature = img[y_exp:y_exp + h_exp, x_exp:x_exp + w_exp]
            cv2.imwrite(f"{freehand_dir}/signature_{freehand_count}.png", signature)
            freehand_count += 1

    print(f"🖋️ Freehand signatures extracted: {freehand_count}")

    # Boxed Signature Detection
    edges = cv2.Canny(blur, 50, 150)
    box_contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boxed_count = 0
    for cnt in box_contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)

        if len(approx) == 4 and cv2.isContourConvex(approx):
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)

            if 50 < w < 300 and 50 < h < 300 and 0.5 < aspect_ratio < 2.0:
                cropped = img[y:y+h, x:x+w]
                cropped_padded = pad_image(cropped, padding=20)
                cv2.imwrite(f"{boxed_dir}/signature_{boxed_count}.png", cropped_padded)
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                boxed_count += 1

    print(f"📦 Boxed signatures extracted: {boxed_count}")

    # Save Final Image
    output_image_path = "combined_signatures_marked.png"
    cv2.imwrite(output_image_path, img)
    print(f"✅ Output image saved with all bounding boxes: {output_image_path}")
extract_signatures("signature.png")


### **Comparing the Extracting images**

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from keras.saving import register_keras_serializable
import matplotlib.pyplot as plt

# Register your custom layer functions
@register_keras_serializable()
def euclidean_distance(vectors):
    x, y = vectors
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

@register_keras_serializable()
def l1_distance(vectors):
    x, y = vectors
    return K.abs(x - y)

# Image preprocessing function
def preprocess_signature(img_path):
    img = image.load_img(img_path, target_size=(128, 256), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = cv2.adaptiveThreshold(img_array.astype(np.uint8), 255,
                                      cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      cv2.THRESH_BINARY_INV, 11, 2)
    img_array = np.expand_dims(img_array, axis=-1)
    img_array = img_array / 255.0
    return np.expand_dims(img_array, axis=0)

# Verification function
def verify_against_original(original_path, test_folder, model_path, threshold=0.6):
    model = load_model(model_path, custom_objects={
        'euclidean_distance': euclidean_distance,
        'l1_distance': l1_distance
    })

    # Load the original signature for display
    original_display = cv2.imread(original_path)
    if original_display is None:
        print(f"❌ Unable to load original image for display: {original_path}")
        return

    # Preprocess the original signature for prediction
    original_preprocessed = preprocess_signature(original_path)

    # Loop over all extracted signatures in the folder
    test_images = sorted(os.listdir(test_folder))
    print(f"\n🧪 Comparing {len(test_images)} extracted signatures against the original...")

    for filename in test_images:
        test_path = os.path.join(test_folder, filename)
        test_preprocessed = preprocess_signature(test_path)

        # Load the test signature for display
        test_display = cv2.imread(test_path)
        if test_display is None:
            print(f"❌ Unable to load test image for display: {test_path}")
            continue

        plt.figure(figsize=(8, 4))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(original_display, cv2.COLOR_BGR2RGB))
        plt.title("Original Signature")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(test_display, cv2.COLOR_BGR2RGB))
        plt.title(f"Test Signature: {filename}")
        plt.axis("off")

        plt.tight_layout()
        plt.show()


        score = model.predict([original_preprocessed, test_preprocessed])[0][0]
        result = "✅ Matching" if score <= threshold else "❌ Not Matching"

        print(f"{filename}: Similarity Score = {score:.4f} → {result}\n")

original_signature_path = "signature1.png"
model_path = "/content/drive/MyDrive/Data/saved_model/siamese_model2.h5"

verify_against_original(original_signature_path, "freehand_signatures", model_path)

verify_against_original(original_signature_path, "boxed_signatures", model_path)